# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [62]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [96]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
    "journal":{
        "file": "dineshb-new.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    }
}

# #todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist_new = {
    "journal":{
        "file": "dinesh-dblp.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    }
}

In [97]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [116]:
for pubsource in publist:
    parser = bibtex.Parser()

    parser1 = bibtex.Parser()

    bibdata = parser.parse_file(publist[pubsource]["file"])
    bibdata1 = parser1.parse_file(publist_new[pubsource]["file"])

#     print(bibdata1.entries)
    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        b = bibdata.entries[bib_id].fields

        print(f'from entry {bib_id}: \"')
#       print(b)
        
        try:
            pub_year = f'{b["year"]}'

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace(".", "").replace("}","").replace("\\","").replace(" ","-")

            for bib_id1 in bibdata1.entries:
                b1 = bibdata1.entries[bib_id1].fields
                clean_title1 = b1["title"].replace("{", "").replace(".", "").replace("}","").replace("\\","").replace(" ","-")
#                 print("Not Matched"+clean_title + " Matched 1: " + clean_title1 +"\n")
                if (clean_title == clean_title1):
                    print("Matched"+clean_title + " Matched 1: " + clean_title1 +"\n")
                    continue

            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
            html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")

            #Build Citation from text
            citation = ""

            #citation authors - todo - add highlighting for primary author?
            for author in bibdata.entries[bib_id].persons["author"]:
                citation = citation+" "+author.first_names[0]+" "+author.last_names[0]+", "

            #citation title
#             citation = citation + "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + ".\""

            #add venue logic depending on citation type
            venue = publist[pubsource]["venue-pretext"]+b[publist[pubsource]["venuekey"]].replace("{", "").replace("}","").replace("\\","")

#             citation = citation + " " + html_escape(venue)
#             citation = citation + ", " + pub_year + "."

            
            ## YAML variables
            md = "---\ntitle: \""   + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + '"\n'
            
            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True

            md += "\ndate: " + str(pub_date) 

            md += "\nvenue: '" + html_escape(venue) + "'"
            
            url = False
            if "url" in b1.keys():
                print("in url: ")
                if len(str(b1["url"])) > 5:
                    md += "\nlink: '" + b1["url"] + "'"
                    url = True
            
            file_var = False
            if "file" in b.keys():
                if len(str(b["file"])) > 5:
                    md += "\npaperurl: '" + b["file"].replace("'", "") + ".pdf'"
                    file_var = True
            
            
            md += "\ncitation: '" + html_escape(citation) + "'"

            md += "\n---"

            
            ## Markdown description for individual page
#             if note:
#                 md += "\n" + html_escape(b["note"]) + "\n"

#             if url:
#                 md += "\n[Access paper here](" + b1["url"] + "){:target=\"_blank\"}\n" 
#             else:
#                 md += "\nUse [Google Scholar](https://scholar.google.com/scholar?q="+html.escape(clean_title.replace("-","+"))+"){:target=\"_blank\"} for full citation"

            md_filename = os.path.basename(md_filename)
            
            if "type" in b.keys():
                if (str(b["type"])=="long-paper"):
                    with open( "../_publications/" + md_filename, 'w') as f:
                       f.write(md)
                    print(md)
                    print(f'SUCESSFULLY PARSED {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
            
            #print(md)
            
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue


from entry pop00001: "
MatchedSyncScatter:-Enabling-WiFi-like-synchronization-and-range-for-WiFi-backscatter-Communication Matched 1: SyncScatter:-Enabling-WiFi-like-synchronization-and-range-for-WiFi-backscatter-Communication

in url: 
---
title: "SyncScatter: Enabling WiFi like synchronization and range for WiFi backscatter Communication."
collection: publications
permalink: /publication/2021-04-01-SyncScatter-Enabling-WiFi-like-synchronization-and-range-for-WiFi-backscatter-Communication
excerpt: '1 cites: https://scholar.google.com/scholar?oi=bibs\&amp;hl=en\&amp;cites=4284191547603180408'
date: 2021-04-01
venue: 'NSDI'
link: 'https://doi.org/10.1145/2486001'
paperurl: '/files/papers/nsdi21-syncscatter.pdf'
citation: ' M Dunna,  M Meng,  PH Wang,  C Zhang,  PP Mercier,  D Bharadia, '
---
SUCESSFULLY PARSED pop00001: " SyncScatter: Enabling WiFi like synchronization and range fo ... "
from entry pop00002: "
in url: 
---
title: "Towards a wireless force sensor based on wave backscatt